In [167]:
import cv2
import os
from uuid import uuid4
from PIL import Image
from manga_ocr import MangaOcr
from sse_starlette.sse import EventSourceResponse
import json
import numpy as np
import PIL.Image


In [2]:
import PIL.Image

from manga_ocr import MangaOcr

mocr = MangaOcr()
img = PIL.Image.open('/path/to/img')
text = mocr(img)

2024-09-19 16:00:40.472 | INFO     | manga_ocr.ocr:__init__:15 - Loading OCR model from kha-white/manga-ocr-base


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/24.1k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\Trung\anaconda3\envs\torch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/77.5k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/444M [00:00<?, ?B/s]

c:\Users\Trung\anaconda3\envs\torch_gpu\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-09-19 16:01:23.194 | INFO     | manga_ocr.ocr:__init__:31 - Using CPU
2024-09-19 16:01:23.675 | INFO     | manga_ocr.ocr:__init__:38 - OCR ready


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/img'

In [6]:
img = PIL.Image.open('./images/opm_163_1_2_jap.jpg')
text = mocr(img)
print(text)

いいです。今日の朝の夕食は、


In [9]:
img = PIL.Image.open('./images/opm_163_part_jap.png')
text = mocr(img)
print(text)

もちろん、これまでは、


In [4]:
#app = FastAPI()
mocr = MangaOcr()


2024-09-19 16:49:22.490 | INFO     | manga_ocr.ocr:__init__:15 - Loading OCR model from kha-white/manga-ocr-base
c:\Users\Trung\anaconda3\envs\torch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\Trung\anaconda3\envs\torch_gpu\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-09-19 16:49:24.797 | INFO     | manga_ocr.ocr:__init__:31 - Usin

In [174]:
def findSpeechBubbles(imagePath, method):
    output_dir = os.path.join('cropped')
    os.makedirs(output_dir, exist_ok=True)
    # read image
    image = cv2.imread(imagePath)
    # gray scale
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # filter noise
    imageGrayBlur = cv2.GaussianBlur(imageGray, (3, 3), 0)
    if method != 'simple':
        # recognizes more complex bubble shapes
        imageGrayBlurCanny = cv2.Canny(imageGrayBlur, 50, 500)
        binary = cv2.threshold(imageGrayBlurCanny, 235,
                               255, cv2.THRESH_BINARY)[1]
    else:
        # recognizes only rectangular bubbles
        binary = cv2.threshold(imageGrayBlur, 235, 255, cv2.THRESH_BINARY)[1]

    # find contours
    contours = cv2.findContours(
        binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

    # get the list of cropped speech bubbles

    croppedImageList = []
    i = 0
    for contour in contours:

        contour = contour.astype(np.int32)
        rect = cv2.boundingRect(contour)
        [x, y, w, h] = rect

        # filter out speech bubble candidates with unreasonable size
        if w < 500 and w > 40 and h < 500 and h > 40:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            croppedImage = image[y:y+h, x:x+w]

            croppedImageList.append(croppedImage)
            # Save the cropped image
            output_path = os.path.join(output_dir, f"{i}.jpg")
            cv2.imwrite(output_path, croppedImage)
            print(f"Saved cropped image {i}.jpg at {output_path}")
            i += 1


    # Optionally, save the image with rectangles drawn
    annotated_image_path = os.path.join(output_dir, "annotated_image.jpg")
    cv2.imwrite(annotated_image_path, image)
    print(f"Annotated image saved to {annotated_image_path}")
findSpeechBubbles("./images/kaguya_1.jpg","")

Saved cropped image 0.jpg at cropped\0.jpg
Saved cropped image 1.jpg at cropped\1.jpg
Saved cropped image 2.jpg at cropped\2.jpg
Saved cropped image 3.jpg at cropped\3.jpg
Saved cropped image 4.jpg at cropped\4.jpg
Saved cropped image 5.jpg at cropped\5.jpg
Saved cropped image 6.jpg at cropped\6.jpg
Saved cropped image 7.jpg at cropped\7.jpg
Saved cropped image 8.jpg at cropped\8.jpg
Saved cropped image 9.jpg at cropped\9.jpg
Saved cropped image 10.jpg at cropped\10.jpg
Saved cropped image 11.jpg at cropped\11.jpg
Saved cropped image 12.jpg at cropped\12.jpg
Saved cropped image 13.jpg at cropped\13.jpg
Saved cropped image 14.jpg at cropped\14.jpg
Saved cropped image 15.jpg at cropped\15.jpg
Saved cropped image 16.jpg at cropped\16.jpg
Saved cropped image 17.jpg at cropped\17.jpg
Saved cropped image 18.jpg at cropped\18.jpg
Saved cropped image 19.jpg at cropped\19.jpg
Saved cropped image 20.jpg at cropped\20.jpg
Saved cropped image 21.jpg at cropped\21.jpg
Saved cropped image 22.jpg at 

Image dimensions: width=1301, height=922
Total contours found: 1706
Contour 0: x=150, y=919, w=9, h=3
Aspect Ratio: 3.00, Extent: 0.02, Contour Area: 0.50
Contour 0 discarded based on aspect ratio or extent.
Contour 1: x=1156, y=917, w=12, h=5
Aspect Ratio: 2.40, Extent: 0.07, Contour Area: 4.00
Contour 1 discarded based on aspect ratio or extent.
Contour 2: x=1151, y=906, w=63, h=16
Aspect Ratio: 3.94, Extent: 0.04, Contour Area: 40.50
Contour 2 discarded based on aspect ratio or extent.
Contour 3: x=1170, y=909, w=9, h=7
Aspect Ratio: 1.29, Extent: 0.44, Contour Area: 27.50
Contour 3 discarded based on aspect ratio or extent.
Contour 4: x=1170, y=908, w=4, h=3
Aspect Ratio: 1.33, Extent: 0.33, Contour Area: 4.00
Contour 4 discarded based on aspect ratio or extent.
Contour 5: x=224, y=900, w=3, h=22
Aspect Ratio: 0.14, Extent: 0.00, Contour Area: 0.00
Contour 5 discarded based on aspect ratio or extent.
Contour 6: x=879, y=896, w=37, h=26
Aspect Ratio: 1.42, Extent: 0.05, Contour Area

In [170]:
img = PIL.Image.open('./cropped/29.jpg')
text = mocr(img)
print(text)

テメーの吸収は技術の無さだ。。。
